# Práctica: Árboles de Decisión (Clasificación)

**Profesor:** Carlos Tessier Fernández
**Módulo:** Sistemas de aprendizaje automático (SAA) 
**Objetivo:** Aplicar un Árbol de Decisión para clasificar especies de pingüinos [RA3, cite: 785].

---

### Contexto

Vamos a utilizar el dataset `Pinguinos.csv`. El objetivo es entrenar un modelo que pueda predecir la **especie** (`species`) de un pingüino basándose en sus características físicas (longitud del pico, longitud de la aleta, masa corporal, etc.) y su ubicación (`island`, `sex`).

**Target (Objetivo):** `species` (Adelie, Chinstrap, Gentoo)
**Features (Características):** Todas las demás columnas.

### Paso 1: Importar Librerías y Cargar Datos

Importa `pandas`, `train_test_split`, `DecisionTreeClassifier`, `accuracy_score`, `confusion_matrix`, `LabelEncoder` y `plot_tree`.

In [ ]:
# HUECO 1.1: Importa todas las librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
# ... (importa el resto)

In [ ]:
# HUECO 1.2: Carga el dataset 'Pinguinos.csv' en un DataFrame llamado 'df'
df = pd.read_csv('Pinguinos.csv')

# HUECO 1.3: Explora los datos (usa .info() y .head())
df.info()
df.head()

### Paso 2: Preprocesamiento de Datos (Limpieza y Codificación)

Como hemos visto con `.info()`, hay valores nulos (NaN) y columnas de tipo `object` (categóricas) que `scikit-learn` no puede procesar directamente.

**Tarea 2.1: Manejar valores nulos**

Por simplicidad, vamos a eliminar cualquier fila que contenga al menos un valor nulo.

In [ ]:
# HUECO 2.1: Elimina las filas con valores nulos (NaN) del DataFrame 'df'
# Pista: usa .dropna()

# Comprueba con df.info() que ya no hay nulos


**Tarea 2.2: Codificar variables categóricas**

Necesitamos convertir las columnas `species` (nuestro target), `island` y `sex` (features) a números.

Ejemplo de uso de ```LabelEncoder```:   
```python
import numpy as np
from sklearn.preprocessing import LabelEncoder

# --- Datos de Ejemplo ---
# Imagina que tenemos datos de una variable categorica (texto).
# 'LabelEncoder' se usa tipicamente para la variable OBJETIVO (la 'y').
# Para las variables FEATURES (la 'X'), es mas comun usar OneHotEncoder,
# pero LabelEncoder tambien se usa a veces si la cardinalidad es baja.

labels = ['Adelie', 'Gentoo', 'Chinstrap', 'Adelie', 'Gentoo']
print(f"Datos Originales: {labels}")
print("---")

# 1. Inicializar el codificador
# Creamos una instancia de la clase
le = LabelEncoder()

# 2. Ajustar y Transformar (fit_transform)
# El metodo .fit_transform() hace dos cosas:
# 1. .fit(): Aprende el mapeo (las clases unicas). Asigna un numero
#            a cada clase por orden alfabetico.
# 2. .transform(): Aplica ese mapeo a los datos.
encoded_labels = le.fit_transform(labels)

print(f"Datos Codificados: {encoded_labels}")
print("---")

# 3. Ver las clases aprendidas
# El atributo .classes_ nos muestra el mapeo.
# El indice del array corresponde al numero asignado.
# (0 -> 'Adelie', 1 -> 'Chinstrap', 2 -> 'Gentoo')
print(f"Clases aprendidas (mapeo): {le.classes_}")
print("---")


# 4. Transformar datos nuevos
# Si aparece un dato que el encoder ya conoce (ej. 'Gentoo')
nuevo_dato = ['Gentoo']
print(f"Dato nuevo: {nuevo_dato}")
print(f"Dato nuevo codificado: {le.transform(nuevo_dato)}")
print("---")

# 5. Invertir la transformacion (inverse_transform)
# Esto es muy util para interpretar las predicciones del modelo.
# Si el modelo predice [2, 0, 1], ¿que especies son?
predicciones_numericas = np.array([2, 0, 1, 0])
predicciones_originales = le.inverse_transform(predicciones_numericas)

print(f"Predicciones (numeros): {predicciones_numericas}")
print(f"Predicciones (texto):   {predicciones_originales}")
```   
OBTENEMOS:    
```
Datos Originales: ['Adelie', 'Gentoo', 'Chinstrap', 'Adelie', 'Gentoo']
---
Datos Codificados: [0 2 1 0 2]
---
Clases aprendidas (mapeo): ['Adelie' 'Chinstrap' 'Gentoo']
---
Dato nuevo: ['Gentoo']
Dato nuevo codificado: [2]
---
Predicciones (numeros): [2 0 1 0]
Predicciones (texto):   ['Gentoo' 'Adelie' 'Chinstrap' 'Adelie']
```   


In [ ]:
# HUECO 2.2.1 (Target): Usa LabelEncoder para convertir la columna 'species'
# 1. Crea una instancia de LabelEncoder
# 2. Ajústala y transfórmala sobre df['species']
# 3. Guarda el resultado en una nueva columna, p.ej. 'species_encoded'

# (Guarda el encoder para referencia futura)


### ONE-HOT ENCODING   
One-Hot Encoding (OHE) es una técnica de preprocesamiento que convierte una columna de datos categóricos en múltiples columnas binarias (0s y 1s).

Cada categoría única en la columna original se convierte en su propia columna nueva, que actúa como un "interruptor" (0 = apagado, 1 = encendido).   

La solución (OHE): OHE crea tres columnas (ej. isla_Torgersen, isla_Biscoe, isla_Dream).

- Un pingüino de Biscoe será: [0, 1, 0]
- Un pingüino de Dream será: [0, 0, 1]
- Un pingüino de Torgersen será: [1, 0, 0]

```python   
# --- Aplicar One-Hot Encoding ---
# pd.get_dummies() detecta automaticamente las columnas de
# texto (tipo 'object') y las convierte.
#
# Nota: 'especie' es nuestro target, pero lo incluimos
# solo para ver como lo codifica.
df_codificado = pd.get_dummies(df)

```
```python    
# --- OHE y la "Trampa del Chupete" (Dummy Variable Trap) ---
#
# Podemos observar que las columnas son redundantes.
# Si 'isla_Biscoe'=0 e 'isla_Dream'=0, ya SABEMOS que 'isla_Torgersen' debe ser 1.
#
# Esta redundancia (multicolinealidad) puede causar problemas
# en algunos modelos (como la Regresion Lineal).
#
# Solucion: Usar 'drop_first=True' para eliminar la primera
# categoria de cada feature, convirtiendola en la 'base' (todo 0s).
df_codificado_optimizado = pd.get_dummies(df, drop_first=True)

```

In [ ]:
# HUECO 2.2.2 (Features): Usa pd.get_dummies para convertir 'island' y 'sex'
# Esto usará One-Hot Encoding, que es lo ideal para features categóricas en árboles.

df_encoded = pd.get_dummies(df, columns=['island', 'sex'], drop_first=True)

# Muestra el nuevo dataframe
df_encoded.head()

### Paso 3: Preparar X (Features) e y (Target)

**Tarea 3.1:** Define `y` como la columna codificada del target (`species_encoded`).
**Tarea 3.2:** Define `X` como las columnas numéricas relevantes (las originales + las creadas por `get_dummies`). Asegúrate de NO incluir `species` (la original) ni `species_encoded` (el target) en `X`.

In [ ]:
# HUECO 3.1: Define 'y'

# HUECO 3.2: Define 'X'
# Pista: puedes hacer X = df_encoded.drop(columns=['species', 'species_encoded'])


### Paso 4: División de Datos (Train/Test)

Divide `X` e `y` en conjuntos de entrenamiento y prueba. Usa un `test_size` de 0.3 y un `random_state` de 42.

In [ ]:
# HUECO 4: Importa train_test_split (si no lo hiciste) y divide los datos


### Paso 5: Entrenamiento del Modelo

Crea una instancia de `DecisionTreeClassifier`. Para evitar overfitting, asígnale un `max_depth` (profundidad máxima) de 4 y un `random_state` de 42. Luego, entrénalo.

In [ ]:
# HUECO 5: Crea el clasificador (clf) y entrénalo (fit)


### Paso 6: Evaluación del Modelo

Usa el modelo entrenado para predecir sobre `X_test`. Calcula el `accuracy_score` y la `confusion_matrix`.

In [ ]:
# HUECO 6: Realiza predicciones (y_pred) y evalúa el modelo


### Paso 7: (Opcional) Visualización

Visualiza el árbol que has entrenado usando `plot_tree`. Necesitarás los nombres de las features (de `X.columns`) y los nombres de las clases (del `LabelEncoder`).

In [ ]:
# HUECO 7 (Opcional): Visualiza el árbol

plt.figure(figsize=(20, 15))
plot_tree(clf,
          filled=True,
          rounded=True,
          class_names=le.classes_, # 'le' es el nombre de tu LabelEncoder
          feature_names=X.columns)
plt.show()